In [73]:
# Librerias
# ==============================================================================
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

import plotly.express as px

## Datos

In [68]:
# Download data Train
# ==============================================================================
url = (r'/home/javi/Downloads/train.csv')
data = pd.read_csv(url, sep=',', header=0)
print(data.info())
data.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB
None


,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14


In [69]:
# Data preprocessing
# ==============================================================================
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.sort_index()

# Train-val-test dates
# ==============================================================================
print(f"Train dates      : {data.index.min()} --- {data.index.max()}  (n={len(data)})")

Train dates      : 2013-01-01 00:00:00 --- 2017-12-31 00:00:00  (n=913000)


In [70]:
# combine store and item column as time_series
# ==============================================================================
data['store'] = ['store_' + str(i) for i in data['store']]
data['item'] = ['item_' + str(i) for i in data['item']]
data['time_series'] = data[['store', 'item']].apply(lambda x: '_'.join(x), axis=1)
data = data.drop(['store', 'item'], axis=1)

# extract features from date
data['month'] = data.index.month
data['year'] = data.index.year
data['day_of_week'] = data.index.day_of_week
data['day_of_year'] = data.index.day_of_year

data.head(3)

,sales,time_series,month,year,day_of_week,day_of_year
date,,,,,,
2013-01-01,13,store_1_item_1,1,2013,1,1
2013-01-01,26,store_7_item_12,1,2013,1,1
2013-01-01,27,store_7_item_46,1,2013,1,1


In [71]:
# check the unique time_series
# ==============================================================================
data['time_series'].nunique()

500

## Exploración Gráfica

In [78]:
# plot time series with moving avg
# ==============================================================================
i = 'store_1_item_1'

subset = data[data['time_series'] == i].copy()
subset['moving_average'] = subset['sales'].rolling(30).mean()
fig = px.line(subset, x=subset.index, y=["sales","moving_average"], title = i, template = 'plotly_dark')
fig.show()

## Test

In [ ]:
# Download data Test
# ==============================================================================
url_test = (r'/home/javi/Downloads/test.csv')
data_test = pd.read_csv(url_test, sep=',', header=0)
print(data_test.info())
data_test.head(3)